# Query Transformations for Enhanced RAG Systems

## Setting Up the Environment

In [1]:
import fitz
import numpy as np
import json

In [2]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Implementing Query Transformation Techniques
### 1. Query Rewriting

In [3]:
def rewrite_query(original_query, model="gpt-4o-mini"):
    """
    검색 정확도를 높이기 위해 쿼리를 더 구체적이고 명확하게 재작성합니다.

    Args:
        original_query (str): 원본 사용자 쿼리.
        model (str): 쿼리 재작성에 사용할 언어 모델.

    Returns:
        str: 재작성된 구체적인 쿼리.
    """
    # AI 어시스턴트의 동작을 안내하는 시스템 프롬프트 정의
    system_prompt = (
        "당신은 검색 쿼리를 개선하는 데 특화된 AI 어시스턴트입니다. "
        "사용자의 원본 쿼리를 더 구체적이고 상세하게 다시 작성하여, "
        "정확한 정보 검색이 가능하도록 돕는 것이 목적입니다."
    )

    # 사용자 프롬프트: 개선이 필요한 원본 쿼리를 포함
    user_prompt = f"""
    다음 쿼리를 더 구체적이고 상세하게 다시 작성하세요. 
    관련된 키워드나 개념을 포함하여 보다 정확한 검색이 가능하도록 만드세요.

    원본 쿼리: {original_query}

    재작성된 쿼리:
    """

    # 지정된 모델을 사용하여 쿼리 재작성 요청
    response = client.chat.completions.create(
        model=model,
        temperature=0.0,  # 결과의 일관성을 위한 낮은 온도 설정
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    # 결과에서 텍스트를 정제하여 반환
    return response.choices[0].message.content.strip()

### 2. Step-back Prompting

In [4]:
def generate_step_back_query(original_query, model="gpt-4o-mini"):
    """
    더 넓은 문맥을 검색할 수 있도록 일반화된 '스텝백(step-back)' 쿼리를 생성합니다.

    Args:
        original_query (str): 원래의 사용자 질문.
        model (str): 스텝백 쿼리 생성을 위한 언어 모델.

    Returns:
        str: 일반화된 스텝백 쿼리.
    """
    # AI 어시스턴트의 동작을 안내하는 시스템 프롬프트
    system_prompt = (
        "당신은 검색 전략에 특화된 AI 어시스턴트입니다. "
        "사용자의 구체적인 질문을 더 일반적이고 포괄적인 질문으로 바꿔, "
        "배경 지식이나 문맥을 넓게 검색할 수 있도록 도와주는 것이 목표입니다."
    )

    # 사용자 프롬프트: 일반화할 원본 쿼리를 포함
    user_prompt = f"""
    다음 질문을 더 넓고 일반적인 형태로 바꾸어,
    관련된 배경 지식이나 문맥 정보를 검색할 수 있도록 하세요.

    원본 쿼리: {original_query}

    스텝백 쿼리:
    """

    # 언어 모델을 사용하여 스텝백 쿼리 생성 요청
    response = client.chat.completions.create(
        model=model,
        temperature=0.1,  # 약간의 다양성을 위한 온도 설정
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    # 결과 반환 (양 끝 공백 제거)
    return response.choices[0].message.content.strip()

### 3. Sub-query Decomposition

In [5]:
def decompose_query(original_query, num_subqueries=4, model="gpt-4o-mini"):
    """
    복잡한 쿼리를 더 단순한 하위 쿼리로 분해합니다.

    Args:
        original_query (str): 복잡한 원본 질문.
        num_subqueries (int): 생성할 하위 질문 수.
        model (str): 쿼리 분해에 사용할 언어 모델.

    Returns:
        List[str]: 단순한 하위 질문 리스트.
    """
    # 시스템 프롬프트: 복잡한 질문을 분해하는 역할
    system_prompt = (
        "당신은 복잡한 질문을 분해하는 데 특화된 AI 어시스턴트입니다. "
        "주어진 질문을 더 단순한 하위 질문들로 나누고, "
        "이 하위 질문들이 함께 원래 질문에 대한 답변을 구성할 수 있도록 하세요."
    )

    # 사용자 프롬프트 정의
    user_prompt = f"""
    다음 복잡한 질문을 {num_subqueries}개의 더 단순한 하위 질문으로 나누세요.
    각 하위 질문은 원래 질문의 서로 다른 측면에 초점을 맞추어야 합니다.

    원본 질문: {original_query}

    다음 형식으로 {num_subqueries}개의 하위 질문을 생성하세요:
    1. [첫 번째 하위 질문]
    2. [두 번째 하위 질문]
    ...
    """

    # 모델을 호출하여 하위 질문 생성
    response = client.chat.completions.create(
        model=model,
        temperature=0.2,  # 약간의 다양성을 허용
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    # 결과에서 질문만 추출
    content = response.choices[0].message.content.strip()
    lines = content.split("\n")
    sub_queries = []

    for line in lines:
        if line.strip() and any(line.strip().startswith(f"{i}.") for i in range(1, 10)):
            # 번호 제거 및 공백 정리
            query = line.strip()
            query = query[query.find(".")+1:].strip()
            sub_queries.append(query)

    return sub_queries

## Demonstrating Query Transformation Techniques

In [7]:
# 예시 쿼리
original_query = "AI가 업무 자동화와 고용에 미치는 영향은 무엇인가요?"

# 쿼리 변환 적용
print("Original Query:")
print(original_query)

# 1. 쿼리 재작성 (더 구체적으로)
rewritten_query = rewrite_query(original_query)
print("\nRewritten Query:")
print(rewritten_query)

# 2. 스텝백 쿼리 생성 (더 일반화된 문맥 요청)
step_back_query = generate_step_back_query(original_query)
print("\nStep-back Query:")
print(step_back_query)

# 3. 하위 쿼리 분해 (다양한 측면으로 나눔)
sub_queries = decompose_query(original_query, num_subqueries=4)
print("\nSub-queries:")
for i, query in enumerate(sub_queries, 1):
    print(f"   {i}. {query}")

Original Query:
AI가 업무 자동화와 고용에 미치는 영향은 무엇인가요?

Rewritten Query:
AI 기술이 업무 자동화에 미치는 영향과 이로 인해 고용 시장에서 발생하는 변화, 특히 특정 산업 분야(예: 제조업, 서비스업, IT업계)에서의 고용 감소 또는 증가 추세, 직무 변화, 새로운 직업의 창출 가능성에 대한 연구 및 사례를 포함하여 설명해 주세요.

Step-back Query:
AI 기술의 발전이 산업 및 노동 시장에 미치는 전반적인 영향은 무엇인가요?

Sub-queries:
   1. AI가 업무 자동화에 어떻게 기여하고 있는가?
   2. AI의 도입이 특정 산업의 고용 구조에 어떤 변화를 가져오는가?
   3. AI로 인해 사라지는 직업과 새롭게 생겨나는 직업은 무엇인가?
   4. AI가 업무 자동화로 인해 직원들의 업무 만족도에 미치는 영향은 무엇인가?


## Building a Simple Vector Store

In [8]:
class SimpleVectorStore:
    """
    NumPy를 활용한 간단한 벡터 저장소 클래스입니다.
    """
    def __init__(self):
        """
        벡터 저장소 초기화.
        """
        self.vectors = []    # 임베딩 벡터를 저장할 리스트
        self.texts = []      # 원본 텍스트를 저장할 리스트
        self.metadata = []   # 각 텍스트에 대한 메타데이터를 저장할 리스트
    
    def add_item(self, text, embedding, metadata=None):
        """
        벡터 저장소에 항목을 추가합니다.

        Args:
            text (str): 원본 텍스트.
            embedding (List[float]): 임베딩 벡터.
            metadata (dict, optional): 추가 메타데이터 (기본값: None).
        """
        self.vectors.append(np.array(embedding))     # 임베딩 벡터를 NumPy 배열로 변환하여 저장
        self.texts.append(text)                      # 원본 텍스트 저장
        self.metadata.append(metadata or {})         # 메타데이터 저장 (없으면 빈 딕셔너리)

    def similarity_search(self, query_embedding, k=5):
        """
        쿼리 임베딩과 가장 유사한 항목들을 검색합니다.

        Args:
            query_embedding (List[float]): 쿼리 임베딩 벡터.
            k (int): 반환할 결과 수 (기본값: 5).

        Returns:
            List[Dict]: 가장 유사한 상위 k개 항목 (텍스트, 메타데이터, 유사도 포함).
        """
        if not self.vectors:
            return []  # 저장된 벡터가 없으면 빈 리스트 반환

        # 쿼리 벡터를 NumPy 배열로 변환
        query_vector = np.array(query_embedding)

        # 코사인 유사도를 계산하여 (인덱스, 유사도) 튜플 저장
        similarities = []
        for i, vector in enumerate(self.vectors):
            similarity = np.dot(query_vector, vector) / (
                np.linalg.norm(query_vector) * np.linalg.norm(vector)
            )
            similarities.append((i, similarity))

        # 유사도를 기준으로 내림차순 정렬
        similarities.sort(key=lambda x: x[1], reverse=True)

        # 상위 k개의 결과 반환
        results = []
        for i in range(min(k, len(similarities))):
            idx, score = similarities[i]
            results.append({
                "text": self.texts[idx],
                "metadata": self.metadata[idx],
                "similarity": score
            })

        return results

## Creating Embeddings

In [10]:
def create_embeddings(text, model="text-embedding-3-small"):
    """
    주어진 텍스트에 대해 지정된 모델을 사용하여 임베딩 벡터를 생성합니다.

    Args:
        text (str or List[str]): 임베딩을 생성할 입력 텍스트 또는 텍스트 리스트.
        model (str): 사용할 임베딩 모델 이름.

    Returns:
        List[float] or List[List[float]]: 단일 텍스트의 경우 임베딩 벡터 하나,
                                          여러 텍스트의 경우 임베딩 벡터 리스트.
    """
    # 입력이 문자열인 경우 리스트로 변환하여 처리
    input_text = text if isinstance(text, list) else [text]
    
    # 지정된 모델로 임베딩 생성 요청
    response = client.embeddings.create(
        model=model,
        input=input_text
    )
    
    # 입력이 문자열이면 첫 번째 임베딩만 반환
    if isinstance(text, str):
        return response.data[0].embedding
    
    # 리스트 입력일 경우 전체 임베딩 벡터 리스트 반환
    return [item.embedding for item in response.data]

## Implementing RAG with Query Transformations

In [11]:
def extract_text_from_pdf(pdf_path):
    """
    PDF 파일에서 텍스트를 추출합니다.

    Args:
        pdf_path (str): PDF 파일의 경로.

    Returns:
        str: 추출된 전체 텍스트 문자열.
    """
    # PDF 파일 열기
    mypdf = fitz.open(pdf_path)
    all_text = ""  # 추출된 텍스트를 저장할 문자열 초기화

    # 각 페이지를 순회하며 텍스트 추출
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]                 # 페이지 객체 가져오기
        text = page.get_text("text")           # 페이지에서 텍스트 추출
        all_text += text                       # 누적하여 전체 텍스트 구성

    return all_text  # 최종 텍스트 반환

In [12]:
def chunk_text(text, n=1000, overlap=200):
    """
    주어진 텍스트를 n자 단위로 중첩(overlap)을 포함하여 분할합니다.

    Args:
        text (str): 분할할 원본 텍스트.
        n (int): 각 청크의 문자 수 (기본값: 1000).
        overlap (int): 청크 간 중첩되는 문자 수 (기본값: 200).

    Returns:
        List[str]: 분할된 텍스트 청크 리스트.
    """
    chunks = []  # 청크를 저장할 리스트 초기화

    # (n - overlap)씩 이동하며 텍스트를 분할
    for i in range(0, len(text), n - overlap):
        chunks.append(text[i:i + n])  # 현재 위치부터 n자까지 잘라 청크로 추가

    return chunks  # 생성된 청크 리스트 반환

In [13]:
def process_document(pdf_path, chunk_size=1000, chunk_overlap=200):
    """
    RAG(Retrieval-Augmented Generation)을 위한 문서 전처리 작업을 수행합니다.

    Args:
        pdf_path (str): PDF 파일 경로.
        chunk_size (int): 각 텍스트 청크의 문자 수.
        chunk_overlap (int): 청크 간 중첩되는 문자 수.

    Returns:
        SimpleVectorStore: 청크와 해당 임베딩이 저장된 벡터 저장소 객체.
    """
    print("PDF에서 텍스트 추출 중...")
    extracted_text = extract_text_from_pdf(pdf_path)

    print("텍스트를 청크 단위로 분할 중...")
    chunks = chunk_text(extracted_text, chunk_size, chunk_overlap)
    print(f"총 {len(chunks)}개의 텍스트 청크가 생성되었습니다.")

    print("청크에 대한 임베딩 생성 중...")
    # 효율성을 위해 모든 청크에 대한 임베딩을 한 번에 생성
    chunk_embeddings = create_embeddings(chunks)

    # 벡터 저장소 생성
    store = SimpleVectorStore()

    # 각 청크와 임베딩을 저장소에 추가
    for i, (chunk, embedding) in enumerate(zip(chunks, chunk_embeddings)):
        store.add_item(
            text=chunk,
            embedding=embedding,
            metadata={"index": i, "source": pdf_path}
        )

    print(f"벡터 저장소에 {len(chunks)}개의 청크가 추가되었습니다.")
    return store

## RAG with Query Transformations

In [14]:
def transformed_search(query, vector_store, transformation_type, top_k=3):
    """
    변환된 쿼리를 사용하여 벡터 저장소에서 검색을 수행합니다.

    Args:
        query (str): 원본 사용자 쿼리.
        vector_store (SimpleVectorStore): 검색 대상 벡터 저장소.
        transformation_type (str): 쿼리 변환 방식 ('rewrite', 'step_back', 'decompose').
        top_k (int): 반환할 상위 결과 수.

    Returns:
        List[Dict]: 검색된 결과 리스트.
    """
    print(f"쿼리 변환 방식: {transformation_type}")
    print(f"원본 쿼리: {query}")

    results = []

    if transformation_type == "rewrite":
        # 쿼리 재작성
        transformed_query = rewrite_query(query)
        print(f"재작성된 쿼리: {transformed_query}")

        # 임베딩 생성 및 검색 수행
        query_embedding = create_embeddings(transformed_query)
        results = vector_store.similarity_search(query_embedding, k=top_k)

    elif transformation_type == "step_back":
        # 스텝백 쿼리 생성
        transformed_query = generate_step_back_query(query)
        print(f"스텝백 쿼리: {transformed_query}")

        # 임베딩 생성 및 검색 수행
        query_embedding = create_embeddings(transformed_query)
        results = vector_store.similarity_search(query_embedding, k=top_k)

    elif transformation_type == "decompose":
        # 복잡한 쿼리를 하위 쿼리로 분해
        sub_queries = decompose_query(query)
        print("하위 쿼리로 분해:")
        for i, sub_q in enumerate(sub_queries, 1):
            print(f"{i}. {sub_q}")

        # 하위 쿼리 각각에 대한 임베딩 생성 및 검색 수행
        sub_query_embeddings = create_embeddings(sub_queries)
        all_results = []

        for i, embedding in enumerate(sub_query_embeddings):
            sub_results = vector_store.similarity_search(embedding, k=2)  # 각 하위 쿼리당 적은 수 반환
            all_results.extend(sub_results)

        # 중복 제거 (동일한 텍스트가 여러 번 등장할 경우, 가장 높은 유사도 결과만 유지)
        seen_texts = {}
        for result in all_results:
            text = result["text"]
            if text not in seen_texts or result["similarity"] > seen_texts[text]["similarity"]:
                seen_texts[text] = result

        # 유사도 기준 내림차순 정렬 후 top_k 개 추출
        results = sorted(seen_texts.values(), key=lambda x: x["similarity"], reverse=True)[:top_k]

    else:
        # 변환 없이 일반 쿼리로 검색
        query_embedding = create_embeddings(query)
        results = vector_store.similarity_search(query_embedding, k=top_k)

    return results

## Generating a Response with Transformed Queries

In [15]:
def generate_response(query, context, model="gpt-4o-mini"):
    """
    쿼리와 검색된 문맥을 기반으로 응답을 생성합니다.

    Args:
        query (str): 사용자 질문.
        context (str): 검색된 문맥 정보.
        model (str): 응답 생성을 위한 언어 모델 이름.

    Returns:
        str: 생성된 응답 문자열.
    """
    # AI 어시스턴트의 동작을 안내하는 시스템 프롬프트 정의
    system_prompt = (
        "당신은 도움이 되는 AI 어시스턴트입니다. 사용자 질문에 대해 "
        "오직 제공된 문맥(Context)만을 기반으로 답변하세요. "
        "만약 문맥에서 답을 찾을 수 없다면, 정보가 부족하다고 솔직하게 말하세요."
    )

    # 사용자 프롬프트 구성: 문맥과 질문 포함
    user_prompt = f"""
        Context:
        {context}

        Question: {query}

        위 문맥에만 근거하여 포괄적이고 명확한 답변을 작성해주세요.
    """

    # 모델을 호출하여 응답 생성
    response = client.chat.completions.create(
        model=model,
        temperature=0,  # 일관된 출력 생성을 위한 낮은 온도
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    # 응답 텍스트 반환
    return response.choices[0].message.content.strip()

## Running the Complete RAG Pipeline with Query Transformations

In [16]:
def rag_with_query_transformation(pdf_path, query, transformation_type=None):
    """
    쿼리 변환을 포함한 RAG 파이프라인 전체를 실행합니다.

    Args:
        pdf_path (str): PDF 문서 경로.
        query (str): 사용자 질문.
        transformation_type (str): 쿼리 변환 방식 (None, 'rewrite', 'step_back', 'decompose').

    Returns:
        Dict: 쿼리, 변환 방식, 검색된 문맥, 생성된 응답을 포함한 결과 딕셔너리.
    """
    # PDF 문서를 처리하여 벡터 저장소 생성
    vector_store = process_document(pdf_path)

    # 쿼리 변환 적용 및 검색
    if transformation_type:
        # 변환된 쿼리를 사용한 검색
        results = transformed_search(query, vector_store, transformation_type)
    else:
        # 변환 없이 일반 쿼리로 검색
        query_embedding = create_embeddings(query)
        results = vector_store.similarity_search(query_embedding, k=3)

    # 검색 결과에서 문맥 추출
    context = "\n\n".join([
        f"PASSAGE {i+1}:\n{result['text']}" for i, result in enumerate(results)
    ])

    # 문맥을 기반으로 응답 생성
    response = generate_response(query, context)

    # 결과 딕셔너리 반환
    return {
        "original_query": query,
        "transformation_type": transformation_type,
        "context": context,
        "response": response
    }

## Evaluating Transformation Techniques

In [17]:
def compare_responses(results, reference_answer, model="gpt-4o-mini"):
    """
    다양한 쿼리 변환 기법을 통해 생성된 응답들을 기준 정답과 비교하여 평가합니다.

    Args:
        results (Dict): 각 쿼리 변환 기법에 대한 결과 딕셔너리 (original, rewrite, step_back, decompose).
        reference_answer (str): 비교 대상이 되는 기준 정답.
        model (str): 평가에 사용할 언어 모델 이름.
    """
    # 평가 시스템용 시스템 프롬프트 정의
    system_prompt = (
        "당신은 RAG 시스템 평가에 특화된 전문가입니다. "
        "다양한 쿼리 변환 기법을 통해 생성된 응답을 기준 정답과 비교하여, "
        "어떤 기법이 가장 정확하고 관련성 있으며 완전한 응답을 생성했는지를 평가하세요."
    )

    # 평가용 텍스트 구성
    comparison_text = f"기준 정답:\n{reference_answer}\n\n"
    for technique, result in results.items():
        comparison_text += f"{technique.capitalize()} 쿼리 응답:\n{result['response']}\n\n"

    # 사용자 프롬프트 구성
    user_prompt = f"""
    {comparison_text}

    각 쿼리 방식 (original, rewrite, step_back, decompose)에 대해 다음을 수행하세요:
    1. 정확성, 완전성, 관련성을 기준으로 1~10 점수 부여
    2. 각 기법의 장점과 단점 기술

    마지막으로 전체 기법을 가장 잘한 순서대로 순위를 매기고,
    어떤 기법이 전반적으로 가장 효과적이었는지 그 이유를 설명하세요.
    """

    # 평가 모델 호출
    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    # 평가 결과 출력
    print("\n***EVALUATION RESULTS***")
    print(response.choices[0].message.content)
    print("-------------------------")

In [18]:
def evaluate_transformations(pdf_path, query, reference_answer=None):
    """
    동일한 쿼리에 대해 다양한 쿼리 변환 기법의 성능을 평가합니다.

    Args:
        pdf_path (str): PDF 문서 경로.
        query (str): 평가할 원본 쿼리.
        reference_answer (str, optional): 기준 정답. 제공되면 비교 평가를 수행합니다.

    Returns:
        Dict: 각 기법별 RAG 결과를 포함한 딕셔너리.
    """
    # 평가할 쿼리 변환 기법 리스트 (None = 원본 쿼리 사용)
    transformation_types = [None, "rewrite", "step_back", "decompose"]
    results = {}

    # 각 기법에 대해 RAG 파이프라인 실행
    for transformation_type in transformation_types:
        type_name = transformation_type if transformation_type else "original"
        print(f"\n***{type_name.upper()} 쿼리로 RAG 실행 중***")

        # 변환 기법에 따라 문서 처리 및 응답 생성
        result = rag_with_query_transformation(pdf_path, query, transformation_type)
        results[type_name] = result

        # 생성된 응답 출력
        print(f"응답 ({type_name} 쿼리):")
        print(result["response"])
        print("=" * 40)

    # 기준 정답이 주어진 경우, 모든 응답을 비교 평가
    if reference_answer:
        compare_responses(results, reference_answer)

    return results

## Evaluation of Query Transformations

In [19]:
# 검증 데이터를 JSON 파일에서 불러옵니다.
with open('dataset/validation.json') as f:
    data = json.load(f)

# 첫 번째 쿼리와 기준 정답을 추출합니다.
query = data[0]['question']
reference_answer = data[0]['ideal_answer']

# PDF 파일 경로 설정
pdf_path = "data/AI_Information.pdf"

# 다양한 쿼리 변환 기법에 대한 RAG 평가 실행
evaluation_results = evaluate_transformations(pdf_path, query, reference_answer)


***ORIGINAL 쿼리로 RAG 실행 중***
PDF에서 텍스트 추출 중...
텍스트를 청크 단위로 분할 중...
총 42개의 텍스트 청크가 생성되었습니다.
청크에 대한 임베딩 생성 중...
벡터 저장소에 42개의 청크가 추가되었습니다.
응답 (original 쿼리):
설명 가능한 AI(Explainable AI, XAI)란 AI 시스템의 결정 과정을 더 투명하고 이해하기 쉽게 만드는 기술을 의미합니다. XAI는 사용자가 AI의 결정이 공정하고 정확한지를 평가할 수 있도록 돕는 데 중점을 두고 있습니다. 이는 AI 시스템이 "블랙 박스"처럼 작동하여 결정 과정이 불투명한 문제를 해결하는 데 중요한 역할을 합니다.

설명 가능한 AI가 중요한 이유는 다음과 같습니다:

1. **신뢰 구축**: AI 시스템의 투명성과 설명 가능성을 높임으로써 사용자들이 AI의 신뢰성과 공정성을 평가할 수 있게 됩니다.
2. **책임과 책임성**: AI 시스템의 결정 과정이 명확해지면, 개발자, 배포자, 사용자 간의 역할과 책임을 정의하는 데 도움이 됩니다.
3. **윤리적 행동**: AI의 결정이 이해 가능해지면, 잠재적인 해악을 다루고 윤리적인 행동을 보장하는 데 기여할 수 있습니다.

결론적으로, 설명 가능한 AI는 AI 시스템의 신뢰성과 책임성을 높이는 데 필수적인 요소입니다.

***REWRITE 쿼리로 RAG 실행 중***
PDF에서 텍스트 추출 중...
텍스트를 청크 단위로 분할 중...
총 42개의 텍스트 청크가 생성되었습니다.
청크에 대한 임베딩 생성 중...
벡터 저장소에 42개의 청크가 추가되었습니다.
쿼리 변환 방식: rewrite
원본 쿼리: '설명 가능한 AI(Explainable AI)'란 무엇이며, 왜 중요한가?
재작성된 쿼리: '설명 가능한 AI(Explainable AI, XAI)'의 정의와 주요 개념, 그리고 이 기술이 인공지능 시스템의 투명성, 신뢰성 및 윤리적 사용에 미치는 중요성에 대해 설명해 주세요. 또한, 설명 가능한 AI의 실제 적